In [9]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np

# ✅ 기본 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 중인 디바이스:", device)

batch_size = 32
num_epochs = 1000
patience = 10  # early stopping patience

# ✅ 카테고리 수 불러오기
categories = np.load("category.npy").tolist()
num_classes = len(categories)

# ✅ 전처리 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# ✅ 커스텀 Dataset 정의
class NpyImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = int(self.labels[idx])
        return img, label

# ✅ 모델 정의 (EfficientNet + 분류기 교체)
def create_model():
    model = models.efficientnet_b0(pretrained=True)
    for param in model.features.parameters():
        param.requires_grad = False
    model.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(model.classifier[1].in_features, 128),
        nn.ReLU(),
        nn.Linear(128, num_classes)
    )
    return model.to(device)

# ✅ 평가 함수 (정확도 + 손실 반환하도록 수정)
def evaluate(model, loader, criterion):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    avg_loss = total_loss / total
    accuracy = correct / total
    return accuracy, avg_loss


# ✅ 전체 학습 루프 (49개 데이터셋 반복)
for i in range(49):
    print(f"\n📦 data{i}.npy 학습 시작...")

    # 🗂 데이터 불러오기
    data_path = f"data/data{i}.npy"
    target_path = f"data/target{i}.npy"
    data = np.load(data_path)  # (N, H, W)
    labels = np.load(target_path)

    # 📏 리사이징 (224x224) → (N, 224, 224)
    resized_data = np.zeros((data.shape[0], 224, 224), dtype=np.uint8)
    for j in range(data.shape[0]):
        resized_data[j] = np.array(Image.fromarray(data[j]).resize((224, 224)))

    # 🔀 train/val/test 분할
    train_imgs, test_imgs, train_labels, test_labels = train_test_split(resized_data, labels, test_size=0.1, random_state=42)
    train_imgs, val_imgs, train_labels, val_labels = train_test_split(train_imgs, train_labels, test_size=0.1, random_state=42)

    # 📦 Dataset / DataLoader 준비 (num_workers=0 반드시!)
    train_ds = NpyImageDataset(train_imgs, train_labels, transform)
    val_ds   = NpyImageDataset(val_imgs, val_labels, transform)
    test_ds  = NpyImageDataset(test_imgs, test_labels, transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0)

    # 🧠 모델 초기화
    model = create_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    # 🏋️‍♀️ 학습
    best_val_acc = 0
    wait = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

        train_loss = running_loss / total
        train_acc = correct / total
        val_acc, val_loss = evaluate(model, val_loader, criterion)

        print(f"Epoch {epoch+1:03d} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best-model.pth")
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("⏹️ Early stopping")
                break

    # ✅ 테스트 성능 확인
    model.load_state_dict(torch.load("best-model.pth"))
    test_acc, test_loss = evaluate(model, test_loader, criterion)
    print(f"✅ 테스트 정확도: {test_acc:.4f}, 테스트 손실: {test_loss:.4f}")

사용 중인 디바이스: cuda

📦 data0.npy 학습 시작...
Epoch 001 | Train Loss: 1.7238, Train Acc: 0.5485 | Val Loss: 1.2526, Val Acc: 0.6622
Epoch 002 | Train Loss: 1.3466, Train Acc: 0.6353 | Val Loss: 1.1963, Val Acc: 0.6802
Epoch 003 | Train Loss: 1.2701, Train Acc: 0.6509 | Val Loss: 1.1447, Val Acc: 0.6899
Epoch 004 | Train Loss: 1.2190, Train Acc: 0.6652 | Val Loss: 1.1309, Val Acc: 0.6993
Epoch 005 | Train Loss: 1.1814, Train Acc: 0.6750 | Val Loss: 1.1184, Val Acc: 0.7043
Epoch 006 | Train Loss: 1.1606, Train Acc: 0.6775 | Val Loss: 1.1057, Val Acc: 0.7101
Epoch 007 | Train Loss: 1.1378, Train Acc: 0.6842 | Val Loss: 1.0979, Val Acc: 0.7125
Epoch 008 | Train Loss: 1.1226, Train Acc: 0.6882 | Val Loss: 1.1238, Val Acc: 0.7101
Epoch 009 | Train Loss: 1.1079, Train Acc: 0.6911 | Val Loss: 1.0980, Val Acc: 0.7121
Epoch 010 | Train Loss: 1.1002, Train Acc: 0.6907 | Val Loss: 1.1127, Val Acc: 0.7109
Epoch 011 | Train Loss: 1.0853, Train Acc: 0.6963 | Val Loss: 1.1253, Val Acc: 0.7077
Epoch 012 | Tra

KeyboardInterrupt: 